In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import xarray as xr
import joblib
import pickle

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def process_reppu_data():
    #path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/REPPU/200/pbig5min.dat' #Ubuntu
    path = r'/home/sachin/Documents/NIPR/Research/Data/REPPU/pbig1min.dat' #Server

    #read the REPPU data
    with open (path) as f:
        rectype = np.dtype(np.float32)
        reppu_data = np.fromfile(f, rectype) #size = 109,900,800

    reppu_data = reppu_data.reshape(-1,30,80)
    reppu_data = reppu_data.mean(axis=1).mean(axis=1) #integrate over the 30x80 grid

    #load MHD dates to match with REPPU data
    mhd_data = pd.read_csv('mhd_dates.csv')
    expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
        for start, end in zip(mhd_data['start'], mhd_data['end'])])
    
    time = np.arange(1440)
    dt = []
    for day in expanded_dt:
        for t in time:
            dt.append(day + pd.Timedelta(minutes=t*1))
    dt = np.array(dt) #convert from list to numpy array
    df = pd.DataFrame(reppu_data, index=dt)
    df = df.rename(columns={0:'pot'})
    df.reset_index(inplace=True)
    df.rename(columns={'index':'dt'}, inplace=True)

    return df

reppu_df = process_reppu_data()
reppu_df   

In [ ]:
def process_omni_and_merge(reppu_df):
    reppu_df = reppu_df.copy()
    
    omni_df = pd.read_csv('omni_mhd_5min.csv')
    omni_df = omni_df.interpolate()
    #omni_df = omni_df.ffill().bfill() #interpolate missing values
    #omni_df = omni_df.dropna() 
    omni_df['dt'] = pd.to_datetime(omni_df['dt'])

    #merge the REPPU and OMNI data
    df = pd.merge(reppu_df, omni_df, on='dt', how='outer')
    #df = df[(df['dt'] >= '2021-12-01') & (df['dt'] <= '2022-01-24')]
    #df = df[(df['dt'] >= '2021-12-01') & (df['dt'] <= '2021-12-02')]
    #df = df[(df['dt'] >= '2021-05-10') & (df['dt'] <= '2021-05-15')] #Kp 7 storm
    df['pot'] = df['pot'] * 1e-3
    #interpolate
    df = df.interpolate()

    return df.reset_index(drop=True)

df = process_omni_and_merge(reppu_df)
df

In [ ]:
def single_feat_df(df, lookback, var):
    df = df.copy()
    #df = df[['dt',var]]

    df.set_index('dt', inplace=True)
    
    for i in range(1, lookback+1):
        #df[f'{var}(t-{i})'] = df[var].shift(i)
        df[f'BY_GSE(t-{i})'] = df['BY_GSE'].shift(i)
        df[f'BZ_GSE(t-{i})'] = df['BZ_GSE'].shift(i)
        df[f'flow_speed(t-{i})'] = df['flow_speed'].shift(i)
        df[f'proton_density(t-{i})'] = df['proton_density'].shift(i)
        df[f'tilt_angle(t-{i})'] = df['tilt_angle'].shift(i)
        
    df.dropna(inplace=True)
    
    return df

feat = 'pot'
df = single_feat_df(df, 6, feat)
df

In [ ]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [ ]:
def scale_split_tensor(df):

    X = df.iloc[:, 1:]
    y = df.iloc[:, 0]

    #scale the data
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    split_index = int(0.8*len(X))#80% train, 20% test

    #split the data
    X_train = X[:split_index]
    X_test = X[split_index:]
    y_train = y[:split_index]
    y_test = y[split_index:]

    #add exta dim for torch reqs
    X_train = X_train.reshape(-1, X.shape[1], 1)
    X_test = X_test.reshape(-1, X.shape[1], 1)
    y_train = y_train.values.reshape(-1, 1)
    y_test = y_test.values.reshape(-1, 1)

    #convert to tensor
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor

X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = scale_split_tensor(df)

In [ ]:
X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 32, 2)
model.to(device)
model

In [ ]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0
    
    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0

    print()

In [ ]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0
    
    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)
    
    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [ ]:
learning_rate = 0.001
num_epochs = 200
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

    #every 5 epochs plot the performance
    with torch.no_grad():
        predicted = model(X_train_tensor.to(device)).to('cpu')
        

In [ ]:
X = df.iloc[:, 1:]
split_index = int(0.8*len(X))#80% train, 20% test
X_train = X[:split_index]

In [ ]:
with torch.no_grad():
    predicted = model(X_train_tensor.to(device)).to('cpu')

plt.figure(figsize=(10, 4))
y_true = y_train_tensor.to('cpu')
R = np.corrcoef(predicted.squeeze(), y_true.squeeze())[0,1]

plt.plot(X_train.index, y_true,  label=f'Actual {feat}')
plt.plot(X_train.index, predicted, label=f'Predicted {feat}')

plt.title(f'Prediction using LSTM from 21/05/10 -- 21/05/14, Epochs: {num_epochs}')
plt.annotate(f'R={R:.2f}', xy=(0.05, 0.1), xycoords='axes fraction', fontsize=12,fontweight='bold')
plt.xlabel(' ')
plt.ylabel(f'Polar Cap Potential [kV]')
plt.legend()
plt.grid()
plt.tight_layout()
#plt.savefig(f'{feat}_lstm_dt.png', dpi=300)
#plt.show()

In [ ]:
def train_and_validate(epochs, model, train_loader, test_loader, optimizer, loss_function, device):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train(True)  # Set the model to train mode
        print(f'Epoch: {epoch + 1}')
        running_loss = 0.0
        
        # Training loop
        for batch_index, batch in enumerate(train_loader):
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_index % 100 == 99:  # print every 100 batches
                avg_loss_across_batches = running_loss / 100
                print('Training Batch {0}, Loss: {1:.3f}'.format(batch_index+1, avg_loss_across_batches))
                running_loss = 0.0
        
        train_losses.append(running_loss / len(train_loader))

        # Validation loop
        model.train(False)  # Set the model to evaluation mode
        running_loss = 0.0
        
        for batch_index, batch in enumerate(test_loader):
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            
            with torch.no_grad():
                output = model(x_batch)
                loss = loss_function(output, y_batch)
                running_loss += loss.item()

        val_losses.append(running_loss / len(test_loader))
        
        print('Validation Loss: {0:.3f}'.format(val_losses[-1]))

    return train_losses, val_losses

#earning_rate = 0.001
#num_epochs = 10
#loss_function = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#train_losses, val_losses = train_and_validate(num_epochs, model, train_loader, test_loader, optimizer, loss_function, device)
